In [1]:
import numpy as np
import matplotlib.pyplot as pyplot

In [15]:
data = np.random.random([1000]) #coming up with arbitrary data and a predictor 
data = np.reshape(data, (100,10)) #reshape to be 100 rows of 10 cols
predict = np.zeros(100)
for i in range(len(data)):
    predict[i] = float(np.random.random()-.5)
    data[i] /= np.sum(data[i]) + predict[i]


In [19]:
class Network:
    #layers is a list of ints. each int represents the number of neurons in that layer.
    #each neuron is linked to every neuron in the previous and next. 
    #the first layer must be the number of columns in the input data
    #the last represents the number of possible outputs
    def __init__(self, layers, learningrate = .01):
        #layers[0] to layers[1] . lets say 4 to 3
        #assume i have a vector of 4 inputs. [-1,1,-1,-2]  also 1 bias [1]
        #this needs to get multiplied by a column vector of [.2,.2,. 2,.2,.2] Transpose. each column represents a neuron in the next                    layer. 
        #so our matrix between layer 0 and layer 1 needs [layers[1]] columns and layers[0] rows. 
        #we'll get slightly better performance if we do the transverse of that and take input as a column so NEURONS * INPUT = OUTPUT
        #so input is a column vector instead and each inbetween-layer is a matrix of layers[n]rows and layers[n-1] columns 
        self.learningrate = learningrate
        self.vals = list()
        self.layers = list()
        for i in range(len(layers)):
            self.vals.append(np.zeros((layers[i],1)) #our list of column vectors to serve as inputs to the next layer. 
            if(i>0):
                self.layers.append(np.ones((layers[n],layers[n-1] + 1))) #our matrix to propagate forward from layer n-1 to layer n.
                #the +1 is for a value to store a bias
        return
    def predict(input):
        return

    def train(input, testvals):
        return

    def test(input, testvals):

1.0015994014628424
1.0038626387317213
1.025040329790093
0.9860316001651155
1.06140662711754
0.905214551373955


In [ ]:
#WIP. using this vid as resource : https://www.youtube.com/watch?v=9RN2Wr8xvro&list=PL-nR3Zo5zPQvaNGqElO9-N-1z-4N94qBi&index=1
#but trying to make it easier to use, more general, commented, and without retarded variable naming conventions